# Models Test Notebook

In this Notebook we will test our 3 models trained on Colab laboratory using GPU

## Model 1:

    * 3 blocs of VGG
    name : 'ownModel_CNN_GPU.h5'
    _____| CNN > CNN > Pool > Dropout
    _____| CNN > CNN > Pool > Dropout
    _____| CNN > CNN > Pool > Dropout
    _____| Flatten > Dense > Dropout & output layer (5 classes)
    _____| optimizer=optimizers.SGD(lr=learning_rate, momentum=0.9, decay=decay_rate, nesterov=False)
    _____| decay_rate = learning_rate / epochs #( learning_rate= 0.001; epochs= 100
    _____| + Data Augmentation
    _____| CNN: Epochs=100, Train accuracy=0.64380, Validation accuracy=0.65200


## Model 2:

    * 3 blocs of VGG different architecture.
    _____| CNN > Pool
    _____| CNN > Pool
    _____| CNN > Pool
    _____| Flatten > Dense > Dropout > Dense > Dropout & output layer (5 classes)
    _____| optimizer=optimizers.Adam()
    _____| + Data Augmentation
    _____| CNN: Epochs=150, Train accuracy=0.66057, Validation accuracy=0.71800
    
## Model 3:

    * Using Transfer learning VGG_16
    _____| CNN > Pool
    _____| CNN > Pool
    _____| CNN > Pool
    _____| Flatten > Dense > Dropout > Dense > Dropout & output layer (5 classes)
    _____| optimizer=optimizers.SGD(lr=learning_rate, momentum=0.9, decay=decay_rate, nesterov=False)
    _____| decay_rate = learning_rate / epochs #( learning_rate= 0.001; epochs= 100
    _____| + Data Augmentation
    _____| CNN: Epochs=100, Train accuracy=1.00000, Validation accuracy=0.96200


# Load Libraries

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [31]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
#from tqdm.notebook import tqdm
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical


In [6]:
!ls '/content/drive/My Drive/dphi_bootcamp'

animal_names.npy			    test_images.npy
model_vgg_transferLearning_GPU100epochs.h5  Testing_set_animals.csv
ownModel_3VGG_CNN_GPU150epochs.h5	    x.npy
ownModel_CNN_GPU.h5			    y.npy
test


In [30]:
# I stored the x & y as .npy for later use on colab for testing.
x = np.load('/content/drive/My Drive/dphi_bootcamp/x.npy')
y = np.load('/content/drive/My Drive/dphi_bootcamp/y.npy')

In [32]:
#Train val split
val_split_num = int(round(0.2*len(y)))

x_train = x[val_split_num:]
y_train = y[val_split_num:]

x_test = x[:val_split_num]
y_test = y[:val_split_num]

print('#########TRAIN###############')
print('x_train', x_train.shape)
print('y_train', y_train.shape)
print('#########TEST###############')
print('x_test', x_test.shape)
print('y_test', y_test.shape)
print('############################')

# OneHotVectorizing categories
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train[:5])

# Input normalization
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print('x_train Shape:', x_train.shape, '\t x_test Shape:',x_train.shape)
print('y_train Shape: ', y_train.shape, '\t y_test Shape:',y_train.shape)
print('image Shape:', x_train[0].shape)


#########TRAIN###############
x_train (2000, 224, 224, 3)
y_train (2000,)
#########TEST###############
x_test (500, 224, 224, 3)
y_test (500,)
############################
[[0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]
x_train Shape: (2000, 224, 224, 3) 	 x_test Shape: (2000, 224, 224, 3)
y_train Shape:  (2000, 5) 	 y_test Shape: (2000, 5)
image Shape: (224, 224, 3)


# Test DATA ( test folder & CSV file)

## Reading the CSV file

In [7]:
test_data = pd.read_csv('/content/drive/My Drive/dphi_bootcamp/Testing_set_animals.csv')

In [8]:
test_data.head()

,filename,target
0,e030b20928e90021d85a5854ee454296eb70e3c818b413...,NaN
1,e030b20929e90021d85a5854ee454296eb70e3c818b413...,NaN
2,e030b2092be90021d85a5854ee454296eb70e3c818b413...,NaN
3,e030b2092ce90021d85a5854ee454296eb70e3c818b413...,NaN
4,e030b2092de90021d85a5854ee454296eb70e3c818b413...,NaN


## Copying the Dataframe

In [9]:
tst = test_data.copy()

## Adding the ./test/ to access the images

In [10]:
tst['filename'] = '/content/drive/My Drive/dphi_bootcamp/test/' + tst['filename']

In [11]:
tst.head()

,filename,target
0,/content/drive/My Drive/dphi_bootcamp/test/e03...,NaN
1,/content/drive/My Drive/dphi_bootcamp/test/e03...,NaN
2,/content/drive/My Drive/dphi_bootcamp/test/e03...,NaN
3,/content/drive/My Drive/dphi_bootcamp/test/e03...,NaN
4,/content/drive/My Drive/dphi_bootcamp/test/e03...,NaN


In [12]:
tst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   filename  910 non-null    object 
 1   target    0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 14.3+ KB


In [13]:
# Function for Resizing and Reshaping the input images
def centering_image(img):
    
    size = [256,256]
    #shape:(h×w×t)taking the first2 elements(h,w) and unpacking them appropriately 
    img_size = img.shape[:2]
    
    # extracting the excess space for centering.
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    
    #creating centered image by taking a 0-matrix and then re-assigning intensities
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized

In [41]:
#!pip install tqdm --upgrade

In [32]:
from skimage import io


In [14]:
# Image Resizing and converting them to array for test images of folder test
""" 
TQDM is a progress bar library. 
Inserting tqdm (or python -m tqdm)between pipes will pass 
through all stdin to stdout while printing progress to stderr
"""
tst_images = []
with tqdm(total=len(tst)) as pbar:
    for i, file_path in enumerate(tst.filename.values):
        
        #read image
        img = cv2.imread(file_path,1)
        
        #color order is changed
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        #resize
        #converting images to square
        if(img.shape[0] > img.shape[1]):
            tile_size = (int(img.shape[1]*256/img.shape[0]),256)
        else:
            tile_size = (256, int(img.shape[0]*256/img.shape[1]))

        #centering the images
        img = centering_image(cv2.resize(img, dsize=tile_size))

        #out put 224*224px 
        img = img[16:240, 16:240]
        tst_images.append(img)
        pbar.update(1)

tst_images = np.array(tst_images)
tst_images = tst_images/255

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [15]:
tst_images.shape


(910, 224, 224, 3)

In [ ]:
# Test Model 1

In [16]:
#model_name = "ownModel_CNN_GPU.h5"
model1 = load_model('/content/drive/My Drive/dphi_bootcamp/ownModel_CNN_GPU.h5')

In [17]:
target_model1 = model1.predict(tst_images)

In [33]:
test_loss1, test_acc1 = model1.evaluate(x_test,  y_test, verbose=2)
print(test_acc1) # x_test and y_test are from the split 

16/16 - 0s - loss: 1.2545 - accuracy: 0.6520
0.6520000100135803


In [18]:
animal_names = np.load('/content/drive/My Drive/dphi_bootcamp/animal_names.npy')

In [20]:
res_target1 = []
real_names1 = []
for i in range(0,910):
    res_target1.append(np.argmax(target_model1[i]))
    real_names1.append(str("'")+animal_names[np.argmax(target_model1[i])]+str("'"))

In [21]:
final_pd1 = pd.DataFrame(real_names1)
final_pd1 = final_pd1.astype(str)
final_pd1.index= tst.index
final_pd1.columns= ['target']
final_pd1.to_csv("/content/drive/My Drive/dphi_bootcamp/model1_ownModel_CNN_GPU_pred.csv",index=False, header=False)

In [ ]:
# Test Model 2

In [22]:
model2 = load_model('/content/drive/My Drive/dphi_bootcamp/ownModel_3VGG_CNN_GPU150epochs.h5')

In [23]:
target_model2 = model2.predict(tst_images)

In [34]:
test_loss2, test_acc2 = model2.evaluate(x_test,  y_test, verbose=2)
print(test_acc2) # x_test and y_test are from the split 

16/16 - 0s - loss: 1.1854 - accuracy: 0.7180
0.7179999947547913


In [24]:
res_target2 = []
real_names2 = []
for i in range(0,910):
    res_target2.append(np.argmax(target_model2[i]))
    real_names2.append(str("'")+animal_names[np.argmax(target_model2[i])]+str("'"))

In [25]:
final_pd2 = pd.DataFrame(real_names2)
final_pd2 = final_pd2.astype(str)
final_pd2.index= tst.index
final_pd2.columns= ['target']
final_pd2.to_csv("/content/drive/My Drive/dphi_bootcamp/model2_ownModel_3VGG_CNN_GPU150epochs_pred.csv",index=False, header=False)

In [ ]:
# Test Model 3

In [26]:
model3 = load_model('/content/drive/My Drive/dphi_bootcamp/model_vgg_transferLearning_GPU100epochs.h5')

In [27]:
target_model3 = model3.predict(tst_images)

In [35]:
test_loss3, test_acc3 = model3.evaluate(x_test,  y_test, verbose=2)
print(test_acc3) # x_test and y_test are from the split 

16/16 - 1s - loss: 0.0702 - accuracy: 0.9620
0.9620000123977661


In [28]:
res_target3 = []
real_names3 = []
for i in range(0,910):
    res_target3.append(np.argmax(target_model3[i]))
    real_names3.append(str("'")+animal_names[np.argmax(target_model3[i])]+str("'"))

In [29]:
final_pd3 = pd.DataFrame(real_names3)
final_pd3 = final_pd3.astype(str)
final_pd3.index= tst.index
final_pd3.columns= ['target']
final_pd3.to_csv("/content/drive/My Drive/dphi_bootcamp/model3_model_vgg_transferLearning_GPU100epochs.csv",index=False, header=False)